In [1]:
from CS2_bo_functions import LHS_Design
from CS2_bo_functions import test_train_split
from CS2_bo_functions import create_y_data
from CS2_bo_functions import calc_y_exp
from CS2_bo_functions import create_sse_data
# from bo_functions_generic import create_y_data
# from bo_functions_generic import calc_y_exp
# from bo_functions_generic import create_sse_data
from scipy.stats import qmc
import torch
import numpy as np
import pandas as pd
import os

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#LHS Design in python - May switch code to use this instead for conciseness
emulator = False
seed = 9
LHS_Points =20
d = 4
# d = 2
sampler = qmc.LatinHypercube(d=d, seed = seed)
sample = sampler.random(n=LHS_Points)

#Only going to regress "a" parameters
l_bounds =  [-2, -2, -10 ,-2]
u_bounds =  [2, 2 , 0, 2 ]

# l_bounds =  [-2, -2]
# u_bounds =  [2, 2 ]
LHS = qmc.scale(sample, l_bounds, u_bounds)

In [3]:
#LHS for experimental data points
exp_LHS_Points = 0 #0 , 10, 15, 20, or 25 for 4, 8, 12, 16, and 24 params
exp_d = 2
exp_sampler = qmc.LatinHypercube(d=exp_d, seed = seed)
exp_sample = exp_sampler.random(n=exp_LHS_Points)

exp_l_bounds =  [-1.5, -0.5]
exp_u_bounds =  [1 , 2]
exp_LHS = qmc.scale(exp_sample, exp_l_bounds, exp_u_bounds)
print(exp_LHS)

[]


In [4]:
#Creates Experimental data and saves to a csv
Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

Theta_True = np.array([-2,2])
# Xexp = np.array([-2,-1,0,1,2])

Xexp = np.array([[-0.558,-0.050,0.623,-0.82, 0.22],
                 [1.442,0.467,0.028,0.62,0.30]])

Xexp = np.vstack((exp_LHS, Xexp.T))
print(Xexp[0])
print(Xexp.shape)


[-0.558  1.442]
(5, 2)


In [5]:
n = Xexp.shape[0]
print(n)
noise_std = 0.01
Yexp = calc_y_exp(Constants, Xexp, noise_std)
# Yexp = calc_y_exp(Theta_True, Xexp, noise_std)
print(Yexp)

df = pd.DataFrame({'X1exp': Xexp[:, 0], 'X2exp': Xexp[:, 1], 'Yexp':Yexp})
# df = pd.DataFrame({'Xexp': Xexp, 'Yexp':Yexp})
print(df)

path = 'Input_CSVs/Exp_Data/d='+str(d)
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv(os.path.join(path,'n='+str(n)+'.csv'))

5
[-146.69947812  -80.76773816 -108.16663897  -40.67072465  -72.22508357]
   X1exp  X2exp        Yexp
0 -0.558  1.442 -146.699478
1 -0.050  0.467  -80.767738
2  0.623  0.028 -108.166639
3 -0.820  0.620  -40.670725
4  0.220  0.300  -72.225084


In [6]:
#Array for Standard Approach
all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4"])
# all_p_standard = pd.DataFrame(LHS, columns = ["theta1", "theta2"])


#Array for Emulator approach
df = all_p_standard.iloc[np.repeat(np.arange(len(all_p_standard)), n)]
df = df.reset_index(drop = True)
Xexp_repeat = np.repeat(Xexp, LHS_Points, axis = 0).reshape(Xexp.shape[1],-1)
# print(Xexp_repeat)
#Add Xexp 
for i in range(Xexp.shape[1]):
    df['X'+str(i+1)+'exp'] = Xexp_repeat[i]    
all_p_emulator = df
# print(all_p_emulator)
# print(all_p_standard)


In [7]:
y_data = create_y_data(all_p_emulator, Constants)
sse_data = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "obj")
# print(y_data)
# print(sse_data)

In [8]:
#Create Training Data and Testing Data
#Set noise parameters and load csv file to generate training data
# all_p = LHS_Design("LHS_Toy_3_Input.csv")

#Create y or SSE data
#Save data
if emulator == False:
    all_2_data = all_p_standard.copy()
    all_2_in_y = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "obj")
    all_2_data['Yexp'] = all_2_in_y
    
    all_2_data_ln_obj = all_p_standard.copy()
    all_2_in_y_ln_obj = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "LN_obj")
    all_2_data_ln_obj['Yexp'] = all_2_in_y_ln_obj
    
    titles_all = np.array(['all_st_data','all_st_ln_obj_data'])
    len_data = str(len(all_2_data))
    all_array = [all_2_data,all_2_data_ln_obj]

else:
    all_3_data = all_p_emulator.copy()
    all_3_in_y = create_y_data(all_p_emulator, Constants)
    all_3_data['Yexp'] = all_3_in_y
    
    titles_all = np.array(['all_emul_data'])
    len_data = str(len(all_3_data))
    all_array = [all_3_data]
    
# print(all_array)

# Save Train Data/Test Data
for i in range(len(titles_all)):
    path = 'Input_CSVs/Train_Data/d=' + str(d) +"/"+ titles_all[i]+"/"
    if not os.path.exists(path):
        os.makedirs(path)
    all_array[i].to_csv(os.path.join(path,'t='+len_data)+'.csv')